<a href="https://colab.research.google.com/github/Oliya-Hu/Oliya-langchain-base/blob/main/langchain_demo_travel_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install langchain_openai langchain langchain_core langchain_community -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.3.60
    Uninstalling langchain-core-0.3.60:
      Successfully uninstalled langchain-core-0.3.60


In [11]:
! pip install docarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 6.6 MB/s eta 0:00:00


In [16]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 5.9 MB/s eta 0:

In [13]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.vectorstores import Chroma
from langchain_community.llms import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.pydantic_v1 import BaseModel, Field
from google.colab import userdata

import os
os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

# 创建 Prompt 模版
template = """
你可以使用如下工具：
{tools}

基于下面的上下文回答问题：
{context}

使用如下的格式生成文本：

Question: 你需要回答的问题
Thought: 你需要思考之后需要执行的操作
Action: 你需要执行的操作，需要在这里的工具中选择[{tool_names}]
Action Input: 调用工具的参数
Observation: 执行工具的结果
... (循环 Thought/Action/Action Input/Observation 的过程 N 次)
Thought: 现在我知道答案了
Final Answer: 最终的回答

让我们开始逐步地思考：
Question: {input}
Thought: {agent_scratchpad}
"""
prompt = PromptTemplate.from_template(template)

# 使用 OpenAI LLM
llm = ChatOpenAI(model="qwen2.5-72b-instruct", temperature=0)

# 定义天气查询工具
class WeatherTool(BaseModel):
    city: str = Field(description="城市名称")
    date : str = Field(description="日期")

@tool("query_weather", args_schema=WeatherTool)
def query_weather(city: str, date: str) -> str:
    """查询天气"""
    # 调用天气查询 API
    return "晴朗"

# 定义日程安排工具
class ScheduleTool(BaseModel):
    city: str = Field(description="日程名称")
    date : str = Field(description="日期")

@tool("schedule", args_schema=ScheduleTool)
def schedule(name: str, date: str) -> bool:
    """安排日程"""
    # 调用日程安排 API
    return True

# 工具列表
tools = [query_weather, schedule]

# 创建 Memory，用于保存对话记录
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 创建 ReAct Agent
agent = create_react_agent(llm, tools, prompt)

# 创建 AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

# 在内存中创建 Vector Store（实际场景可以用真实向量数据库）
loader = WebBaseLoader(
        web_path="https://baike.baidu.com/item/%E8%8B%8F%E5%B7%9E%E5%B8%82/6743370",
    )
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
embedding = OpenAIEmbeddings(model="text-embedding-v2", check_embedding_ctx_length=False, chunk_size=25)
db = Chroma.from_documents(documents=splits, embedding=embedding)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 2})



# 设定 Prompt 参数的获取方式
setup_and_retrieval = RunnableParallel(
    {
        "context": retriever,
        "input": RunnablePassthrough(),
    }
)

# 先经过 Retrieval，再运行 Agent
chain = setup_and_retrieval | agent_executor

# 第一次调用回答问题
chain.invoke("帮我查询苏州这周六的天气，如果天气晴朗，则安排到苏州的旅行；否则帮我安排一个室内活动。")
# >> {'output': '苏州下周六的天气晴朗，已为你安排下周六到苏州旅行的日程。'}

# 第二次调用回答问题
chain.invoke("可以为我做一个当天的旅行攻略吗？我希望能到当地的博物馆参观。")
# >> {'output': '以下是为你准备的苏州旅行攻略：...'}



> Entering new AgentExecutor chain...
首先，我需要查询苏州这周六的天气情况。
Action: query_weather
Action Input: {"city": "苏州", "date": "周六"}

ValidationError: 1 validation error for WeatherTool
date
  field required (type=value_error.missing)

In [9]:
print(prompt)

input_variables=['agent_scratchpad', 'context', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} template='\n你可以使用如下工具：\n{tools}\n\n基于下面的上下文回答问题：\n{context}\n\n使用如下的格式生成文本：\n\nQuestion: 你需要回答的问题\nThought: 你需要思考之后需要执行的操作\nAction: 你需要执行的操作，需要在这里的工具中选择[{tool_names}]\nAction Input: 调用工具的参数\nObservation: 执行工具的结果\n... (循环 Thought/Action/Action Input/Observation 的过程 N 次)\nThought: 现在我知道答案了\nFinal Answer: 最终的回答\n\n让我们开始逐步地思考：\nQuestion: {input}\nThought: {agent_scratchpad}\n'
